In [3]:
import sys
!{sys.executable} -m pip install torch

    83% |██████████████████████████▉     | 696.3MB 26.7MB/s eta 0:00:06

    83% |██████████████████████████▉     | 698.0MB 53.0MB/s eta 0:00:03

    84% |███████████████████████████     | 699.8MB 36.9MB/s eta 0:00:04

    84% |███████████████████████████     | 701.5MB 27.4MB/s eta 0:00:05

    84% |███████████████████████████▏    | 705.1MB 51.8MB/s eta 0:00:03

...

    86% |███████████████████████████▋    | 716.6MB 30.1MB/s eta 0:00:04

    86% |███████████████████████████▊    | 720.6MB 110.1MB/s eta 0:00:02

    86% |███████████████████████████▉    | 722.2MB 31.8MB/s eta 0:00:04

    86% |███████████████████████████▉    | 723.3MB 25.4MB/s eta 0:00:05

    100% |████████████████████████████████| 831.4MB 93kB/s  eta 0:00:011   87% |████████████████████████████    | 726.6MB 20.4MB/s eta 0:00:06    88% |████████████████████████████▎   | 734.1MB 14.2MB/s eta 0:00:07    88% |████████████████████████████▎   | 735.3MB 11.4MB/s eta 0:00:09    88% |████████████████████████████▌   | 739.6MB 14.1MB/s eta 0:00:07    89% |████████████████████████████▋   | 742.7MB 17.4MB/s eta 0:00:06    89% |████████████████████████████▋   | 743.6MB 16.0MB/s eta 0:00:06    89% |████████████████████████████▉   | 748.3MB 16.6MB/s eta 0:00:06    90% |█████████████████████████████   | 751.5MB 19.9MB/s eta 0:00:05    90% |█████████████████████████████   | 756.1MB 22.3MB/s eta 0:00:04    91% |█████████████████████████████▏  | 758.7MB 34.9MB/s eta 0:00:03    91% |█████████████████████████████▎  | 760.0MB 26.6MB/s eta 0:00:03    91% |█████████████████████████████▎  | 761.4MB 31.0MB/s eta 0:00:03    93% |██████████████████████████████  | 778.1MB 46.5MB/s eta 0:00:02    94

In [1]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
import torch
import json
from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample,
    UniformCropVideo
)
from typing import Dict

ModuleNotFoundError: No module named 'torch'

In [ ]:
import wget

wget.download('https://dl.fbaipublicfiles.com/pytorchvideo/projects/archery.mp4')


with open("kinetics_classnames.json", "r") as f:
    kinetics_classnames = json.load(f)

# Create an id to label name mapping
kinetics_id_to_classname = {}
for k, v in kinetics_classnames.items():
    kinetics_id_to_classname[v] = str(k).replace('"', "")

In [92]:
# Device on which to run the model
# Set to cuda to load on GPU
device = "cpu"

# Pick a pretrained model and load the pretrained weights
model_name = "slowfast_r50"
model = torch.hub.load("facebookresearch/pytorchvideo", model=model_name, pretrained=True)

# Set to eval mode and move to desired device
model = model.to(device)
model = model.eval()

Using cache found in /home/ubuntu/snap/jupyter/6/.cache/torch/hub/facebookresearch_pytorchvideo_master


# Fine-tune feature extraction

In [110]:
import os
import requests

import torch.nn as nn

feature_extract = True
num_classes = 51

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [111]:
# Input required by the modelssssss

# define function where I can initialize each of the model we might want to use
model.blocks[6].proj = nn.Linear(2304, num_classes)

In [112]:
####################
# SlowFast transform
####################

# We might want different options for transforming the data depending on the model we have

side_size = 256
mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
crop_size = 256
num_frames = 32
sampling_rate = 2
frames_per_second = 30
alpha = 4

class PackPathway(torch.nn.Module):
    """
    Transform for converting video frames as a list of tensors.
    """
    def __init__(self):
        super().__init__()

    def forward(self, frames: torch.Tensor):
        fast_pathway = frames
        # Perform temporal sampling from the fast pathway.
        slow_pathway = torch.index_select(
            frames,
            1,
            torch.linspace(
                0, frames.shape[1] - 1, frames.shape[1] // alpha
            ).long(),
        )
        frame_list = [slow_pathway, fast_pathway]
        return frame_list

transform =  ApplyTransformToKey(
    key="video",
    transform=Compose(
        [
            UniformTemporalSubsample(num_frames),
            Lambda(lambda x: x/255.0),
            NormalizeVideo(mean, std),
            ShortSideScale(
                size=side_size
            ),
            CenterCropVideo(crop_size),
            PackPathway()
        ]
    ),
)

# The duration of the input clip is also specific to the model.
clip_duration = (num_frames * sampling_rate)/frames_per_second

# Data

In [104]:
url = 'https://api-dev.lollllz.com'
secret = 'a5aed2a8-db80-4b22-bf72-11f2d0765572'
headers = {'user_secret': secret, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

# This is a convenience for development purposes, IN EVAL ALWAYS USE `full`
data_type = 'sample' # can either be `sample` or `full`

r = requests.post(f"{url}/auth/create_session", json={'session_name': 'testing', 'data_type': data_type, 'task_id': 'problem_test_video_classification'}, headers=headers)
r.json()
session_token = r.json()['session_token']

In [105]:
headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

r = requests.get(f"{url}/seed_labels", headers=headers)
print(json.dumps(r.json(), indent = 4))

{
    "Labels": [
        {
            "class": "brush_hair",
            "end_frame": 15215,
            "id": "154",
            "start_frame": 14941
        },
        {
            "class": "cartwheel",
            "end_frame": 4049,
            "id": "44",
            "start_frame": 3970
        },
        {
            "class": "catch",
            "end_frame": 1108,
            "id": "13",
            "start_frame": 1080
        },
        {
            "class": "chew",
            "end_frame": 176,
            "id": "1",
            "start_frame": 54
        },
        {
            "class": "clap",
            "end_frame": 3646,
            "id": "41",
            "start_frame": 3332
        },
        {
            "class": "climb",
            "end_frame": 15366,
            "id": "156",
            "start_frame": 15252
        },
        {
            "class": "climb_stairs",
            "end_frame": 3969,
            "id": "43",
            "start_frame": 3892
        },


In [109]:
r_data = requests.get(f"{url}/dataset_metadata/hmdb", headers=headers)
print(len(r_data.json()['dataset_metadata']['classes']))

51


In [118]:
r_task = requests.get(f"{url}/task_metadata/problem_test_video_classification", headers=headers)
print(r_task.json())

{'task_metadata': {'adaptation_dataset': 'hmdb', 'adaptation_evaluation_metrics': ['accuracy'], 'adaptation_label_budget_full': [51, 102, 204, 408, 779, 1486, 2836, 5412], 'adaptation_label_budget_sample': [51, 102, 204, 408, 485, 577, 686, 816], 'base_dataset': 'hmdb', 'base_evaluation_metrics': ['accuracy'], 'base_label_budget_full': [51, 102, 204, 408, 779, 1486, 2836, 5412], 'base_label_budget_sample': [51, 102, 204, 408, 485, 577, 686, 816], 'problem_type': 'video_classification', 'task_id': 'problem_test_video_classification', 'uda_adapt_to_base_overlap_ratio': 1.0, 'uda_base_to_adapt_overlap_ratio': 1.0, 'whitelist': ['imagenet_1k']}}


In [155]:
import numpy as np
root = '../../../../../lwll/development/hmdb/'

unlabeled_image_path = '../../../../../lwll/development/hmdb/hmdb_sample/train'
video = True

In [190]:
import os

from torch.utils.data import Dataset
from PIL import Image
import torch
import torchvision.transforms as transforms

class CustomImageDataset(Dataset):
    """
    A custom dataset used to create dataloaders.
    """
    def __init__(self, filepaths, labels=None, label_map=None, transform=None):
        """
        Create a new CustomImageDataset.
        
        :param filepaths: A list of filepaths. 
        :param labels: A list of labels
        :param label_map: A dictionary to map string labels to intergers
        :param transform: A transform to perform on the images
        """
        self.filepaths = filepaths
        self.labels = labels
        self.label_map = label_map
        self.transform = transform

    def __getitem__(self, index):
        img = Image.open(self.filepaths[index]).convert('RGB')

        if self.transform is not None:
            img = self.transform(img)

        if self.labels is not None:
            if self.label_map is not None:
                label = torch.tensor(self.label_map[(self.labels[index])])
            else:
                label = torch.tensor(int(self.labels[index]))
            return img, label
        else:
            return img

    def __len__(self):
        return len(self.filepaths)
    

class CustomVideoDataset(Dataset):
    """
        A custom dataset used to create dataloaders.
        """
    
    def __init__(self, filepaths, labels=None, label_map=None, transform=None, clips_dictionary=None):
        """
        Create a new CustomVideoDataset.
        :param filepaths: A list of filepaths.
        :param labels: A list of labels
        :param label_map: A dictionary to map string labels to intergers
        :param transform: A transform to perform on the frames
        :pram clips_dictionary: dictionary (id clip, list images) to get frames of a clip
        """
        self.filepaths = filepaths
        self.labels = labels
        self.label_map = label_map
        self.transform = transform
        self.clips_dictionary = clips_dictionary
    
    def __getitem__(self, index):
        clip_id = str(os.path.basename(self.filepaths[index]))#int(os.path.basename(self.filepaths[index]))  # chech what path you have/want
        frames_paths = self.clips_dictionary[clip_id]
        # print(f"FRAMES list[:2]: {frames_paths[:2]} and number of frames {len(frames_paths)}")
        
        frames = []
        for f in frames_paths:  # get same size clips - random pick for eval
            frame = Image.open(f).convert('RGB')
            if self.transform is not None:  # BE CAREFUL TRANSFORMATION MIGHT NEED TO CHANGE FOR VIDEO EVAL!!!!!
                frame = self.transform(frame)
            frames.append(frame)
        
        img = torch.stack(frames)  # need to be of the same size!
        
        if self.labels is not None:
            if self.label_map is not None:
                label = torch.tensor(self.label_map[(self.labels[index])])
            else:
                label = torch.tensor(int(self.labels[index]))
            return img, label
        else:
            return img
    
    def __len__(self):
        return len(self.filepaths)


class SoftLabelDataset(Dataset):
    """
    A custom dataset used to create dataloaders.
    """
    def __init__(self, dataset, labels, remove_old_labels=False):
        """
        Create a new SoftLabelDataset.
        :param dataset: A PyTorch dataset
        :param labels: A list of labels
        :param remove_old_labels: A boolean indicating whether to the dataset returns labels that we do not use
        """
        self.dataset = dataset
        self.labels = labels
        self.remove_old_labels = remove_old_labels

    def __getitem__(self, index):
        data = self.dataset[index]
        label = self.labels[index]
        
        if self.remove_old_labels:
            data = data[0]
            
        return data, label

    def __len__(self):
        return len(self.dataset)
    
def transform_image(train=True):
    """
    Get the transform to be used on an image.
    :return: A transform
    """
    data_mean = [0.485, 0.456, 0.406]
    data_std = [0.229, 0.224, 0.225]
    # Remember to check it for video and eval
    if train:
        return transforms.Compose([
            transforms.RandomResizedCrop((224, 224), scale=(0.8, 1.0)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=data_mean, std=data_std)
        ])
    else:
        return transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=data_mean, std=data_std)
        ])

In [172]:
import pandas as pd
data_type = 'sample'
#train_meta = pd.read_feather('/lwll/development/hmdb/labels_sample')#(f"{root}labels_{data_type}/meta_train.feather")
#train_ids = train_meta['id'].tolist()

In [198]:
labels = r.json()['Labels']

labels_list = []
dictionary_clips = {}
for clip in labels:
    action_frames = [str(clip['id']) + '/' + str(i)+'.jpg' for i in range(clip['start_frame'], clip['end_frame'])]
    dictionary_clips[clip["id"]] = action_frames
    labels_list.append([clip["class"], clip["id"]])

image_labels, image_names  = list(zip(*labels_list))

image_paths = [os.path.join(unlabeled_image_path, str(image_name)) for image_name in image_names]



if video:
    paths_dictionary_clips = {}
    for clip, frames in dictionary_clips.items():
        paths_dictionary_clips[clip] = [os.path.join(unlabeled_image_path, str(f)) for f in frames]
    dictionary_clips = paths_dictionary_clips
else:
    dictionary_clips = None

image_paths = np.asarray(image_paths)
image_labels = np.asarray(image_labels)

In [199]:
checkpoint_num = 1

if checkpoint_num >= 4:
    # 80% for training, 20% for validation
    train_percent = 0.8
    num_data = len(image_paths)
    indices = list(range(num_data))
    train_split = int(np.floor(train_percent * num_data))
    np.random.shuffle(indices)
    train_idx = indices[:train_split]
    val_idx = indices[train_split:]
else:
    train_idx = list(range(len(image_paths)))
    val_idx = []

In [200]:
label_map = {}
class_names = r_data.json()['dataset_metadata']['classes']
for idx, item in enumerate(class_names):
    label_map[item] = idx

label_map = label_map

In [201]:
if video:
    train_dataset = CustomVideoDataset(image_paths[train_idx],
                                       labels=image_labels[train_idx],
                                       label_map=label_map,
                                       transform=transform_image(train=True),
                                       clips_dictionary=dictionary_clips)
    if len(val_idx) != 0:
        val_dataset = CustomVideoDataset(image_paths[val_idx],
                                         labels=image_labels[val_idx],
                                         label_map=label_map,
                                         transform=transform_image(train=False),
                                         clips_dictionary=dictionary_clips)
    else:
        val_dataset = None

In [203]:
train_dataset[0]

FileNotFoundError: [Errno 2] No such file or directory: '../../../../../lwll/development/hmdb/hmdb_sample/train/154/14941.jpg'

In [ ]:
####################
# SlowFast transform
####################

# We might want different options for transforming the data depending on the model we have

side_size = 256
mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
crop_size = 256
num_frames = 32
sampling_rate = 2
frames_per_second = 30
alpha = 4

class PackPathway(torch.nn.Module):
    """
    Transform for converting video frames as a list of tensors.
    """
    def __init__(self):
        super().__init__()

    def forward(self, frames: torch.Tensor):
        fast_pathway = frames
        # Perform temporal sampling from the fast pathway.
        slow_pathway = torch.index_select(
            frames,
            1,
            torch.linspace(
                0, frames.shape[1] - 1, frames.shape[1] // alpha
            ).long(),
        )
        frame_list = [slow_pathway, fast_pathway]
        return frame_list

transform =  ApplyTransformToKey(
    key="video",
    transform=Compose(
        [
            UniformTemporalSubsample(num_frames),
            Lambda(lambda x: x/255.0),
            NormalizeVideo(mean, std),
            ShortSideScale(
                size=side_size
            ),
            CenterCropVideo(crop_size),
            PackPathway()
        ]
    ),
)

# The duration of the input clip is also specific to the model.
clip_duration = (num_frames * sampling_rate)/frames_per_second

In [221]:
# Load the example video
video_path = "archery.mp4"

# Select the duration of the clip to load by specifying the start and end duration
# The start_sec should correspond to where the action occurs in the video
start_sec = 0
end_sec = start_sec + clip_duration


In [222]:
# Initialize an EncodedVideo helper class
video = EncodedVideo.from_path(video_path)

In [223]:
video.__dict__

{'_video_name': 'archery.mp4',
 '_decode_audio': True,
 '_container': <av.InputContainer <_io.BytesIO object at 0x7f57f7973fc0>>,
 '_video_time_base': Fraction(1, 30000),
 '_video_start_pts': 0,
 '_has_audio': (<av.AudioStream #1 aac at 44100Hz, stereo, fltp at 0x7f57f7a086a8>,),
 '_audio_time_base': Fraction(1, 44100),
 '_audio_start_pts': 265,
 '_video': None,
 '_audio': None,
 '_selective_decoding': True,
 '_duration': 10.01}

In [224]:
video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)

In [225]:
video_data['video'].size()

torch.Size([3, 64, 240, 320])

In [219]:
# Apply a transform to normalize the video input
video_data = transform(video_data)

In [220]:
video_data['video'][0].size()

torch.Size([3, 8, 256, 256])

# Prendere una clip e passarla dentro 

### Prendi con API

In [63]:
model

Net(
  (blocks): ModuleList(
    (0): MultiPathWayWithFuse(
      (multipathway_blocks): ModuleList(
        (0): ResNetBasicStem(
          (conv): Conv3d(3, 64, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
          (norm): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activation): ReLU()
          (pool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=[0, 1, 1], dilation=1, ceil_mode=False)
        )
        (1): ResNetBasicStem(
          (conv): Conv3d(3, 8, kernel_size=(5, 7, 7), stride=(1, 2, 2), padding=(2, 3, 3), bias=False)
          (norm): BatchNorm3d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activation): ReLU()
          (pool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=[0, 1, 1], dilation=1, ceil_mode=False)
        )
      )
      (multipathway_fusion): FuseFastToSlow(
        (conv_fast_to_slow): Conv3d(8, 16, kernel_size=(7, 1, 1), st

In [44]:
# Move the inputs to the desired device
inputs = video_data["video"]
inputs = [i.to(device)[None, ...] for i in inputs]

In [45]:
# Pass the input clip through the model
preds = model(inputs)

In [226]:
label_map

{'talk': 0,
 'climb': 1,
 'shoot_bow': 2,
 'ride_bike': 3,
 'pullup': 4,
 'ride_horse': 5,
 'smile': 6,
 'climb_stairs': 7,
 'brush_hair': 8,
 'pushup': 9,
 'walk': 10,
 'drink': 11,
 'run': 12,
 'pick': 13,
 'somersault': 14,
 'hug': 15,
 'fencing': 16,
 'sit': 17,
 'catch': 18,
 'shake_hands': 19,
 'hit': 20,
 'kiss': 21,
 'situp': 22,
 'dive': 23,
 'sword': 24,
 'wave': 25,
 'sword_exercise': 26,
 'flic_flac': 27,
 'throw': 28,
 'punch': 29,
 'swing_baseball': 30,
 'stand': 31,
 'shoot_ball': 32,
 'laugh': 33,
 'kick_ball': 34,
 'dribble': 35,
 'cartwheel': 36,
 'turn': 37,
 'handstand': 38,
 'shoot_gun': 39,
 'draw_sword': 40,
 'kick': 41,
 'smoke': 42,
 'clap': 43,
 'chew': 44,
 'fall_floor': 45,
 'pour': 46,
 'eat': 47,
 'push': 48,
 'jump': 49,
 'golf': 50}

In [227]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [231]:
model.blocks[6].proj.in_features


2304

In [ ]:
from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)
from pytorchvideo.data.encoded_video import EncodedVideo

from typing import Dict

In [ ]:
import pytorchvideo.transforms as video_transform

In [ ]:
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample,
    UniformCropVideo
)

In [ ]:
side_size = 256
mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
crop_size = 256
num_frames = 32
sampling_rate = 2
frames_per_second = 30
alpha = 4

class PackPathway(torch.nn.Module):
    """
    Transform for converting video frames as a list of tensors.
    """
    def __init__(self):
        super().__init__()

    def forward(self, frames: torch.Tensor):
        fast_pathway = frames
        # Perform temporal sampling from the fast pathway.
        slow_pathway = torch.index_select(
            frames,
            1,
            torch.linspace(
                0, frames.shape[1] - 1, frames.shape[1] // alpha
            ).long(),
        )
        frame_list = [slow_pathway, fast_pathway]
        return frame_list

def transform_video(self):   
    """Trasformation valid for SlowFast"""
    side_size = 256
    mean = [0.45, 0.45, 0.45]
    std = [0.225, 0.225, 0.225]
    crop_size = 256
    num_frames = 32
    sampling_rate = 2
    frames_per_second = 30
    alpha = 4

    return  ApplyTransformToKey(key="video",
                                transform=transforms.Compose([
                                    UniformTemporalSubsample(num_frames),
                                    transforms.Lambda(lambda x: x/255.0),
                                    NormalizeVideo(mean, std),
                                    ShortSideScale(size=side_size),
                                    CenterCropVideo(crop_size),
                                    PackPathway()
                                ])
                                )

In [ ]:
def transform_image(train=True):
    """
    Get the transform to be used on an image.
    :return: A transform
    """
    data_mean = [0.45, 0.45, 0.45]
    data_std = [0.225, 0.225, 0.225]
    # Remember to check it for video and eval
    if train:
        return transforms.Compose([
            #transforms.RandomResizedCrop((224, 224), scale=(0.8, 1.0)),
            #transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            #transforms.Normalize(mean=data_mean, std=data_std)
        ])
    else:
        return transforms.Compose([
            #transforms.Resize((224, 224)),
            transforms.ToTensor(),
            #transforms.Normalize(mean=data_mean, std=data_std)
        ])